In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

## Generate list of names from VGGFace2 Class IDs

In [2]:
import pandas as pd
id_meta = pd.read_csv("./data/vggface2/identity_meta.csv", sep="\n")
id_meta = id_meta['Class_ID, Name, Sample_Num, Flag, Gender'].str.split(',', expand=True)

id_meta.columns = ['Class_ID', 'Name', 'Sample_Num', 'Flag', 'Gender', 'None']
id_meta.drop(columns=['None'], inplace=True)

vgg_names = id_meta.drop(columns=['Sample_Num', 'Flag', 'Gender']).set_index('Class_ID')

In [3]:
def get_name(class_id, names=vgg_names,):
    return eval(names['Name'][class_id])

## Setup PyTorch to run on GPU

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


# Instantiate models

### MTCNN provides us with a face-detection pipeline. 
#### This allows us to upload images that are not strictly only of faces (so a full-body image, for example). It also does some light pre-processing on the image before it is sent to the actual classifier.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

### ResNet is our actual classifier.

In [6]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
resnet.classify = True

In [7]:
from PIL import Image

In [8]:
def classify(path):
    img = Image.open(path)

    # crop and pre-whiten image tensor
    img_cropped = mtcnn(img).to(device)

    img_probs = resnet(img_cropped.unsqueeze(0))

    result = img_probs[0].argmax()
    index = result.item() + 1
    return get_name(class_id=index, names=vgg_names)

#### Some simple testing

In [9]:
classify('./data/afinefrenzy.jpg')

'A_Fine_Frenzy'

In [10]:
classify('./data/ajbuckley.jpg')

'A._J._Buckley'